In [267]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

import datetime

In [268]:
df_train = pd.read_csv('data/trainData.csv')
df_sample = pd.read_csv('data/sample.csv')

In [269]:
df_train.info()
df_sample.info()
# df_sample['地区'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16305 entries, 0 to 16304
Data columns (total 5 columns):
日期    16305 non-null object
地区    16305 non-null object
价格    16305 non-null float64
数量    16305 non-null float64
均重    16305 non-null float64
dtypes: float64(3), object(2)
memory usage: 637.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1305 entries, 0 to 1304
Data columns (total 3 columns):
﻿日期    1305 non-null object
地区     1305 non-null object
价格     1305 non-null float64
dtypes: float64(1), object(2)
memory usage: 30.7+ KB


In [270]:
df_train['地区'].value_counts()
# df_train.head()
# df_train.tail()

df_train_GD = df_train[df_train['地区'] == '广东']

In [271]:
def autoRegression(sequence, p, k=7):
    '''
    sequence ：基于时间顺序的序列
    p ：假设的参数个数，这里给出p=7
    coeff： 得到的参数
    k: 验证的个数
    检查p是否可以接受
    这里通过比较p之后的k个值，得到的预测值，是否与真实值的误差都在10^-2之内
    如果都满足就接受，否则就不接受,
    如果可以接受，就预测序列之后的值

    '''
    if len(sequence) < 2*p: return None

    x = []
    b = []
    for i in range(p):
        tmp = sequence[i:p+i]
        x.append(tmp)
    b = sequence[p:2*p]

    x = np.array(x)
    b = np.array(b)

    try: 
        if np.linalg.det(x)==0:
            return None
    except TypeError:
        return None

    coeff = np.linalg.inv(x).dot(b)
#     print(coeff)
    k = min(k, len(sequence) - 2*p)

    for i in range(k):
        t = 2*p + i
        predict = np.sum(coeff * sequence[t-p:t])
#         print(t,predict)
        if np.abs(predict - sequence[t]) > 1:
            return None


    nextvalue = np.sum(sequence[len(sequence)-p:] * coeff)

    return nextvalue


In [272]:
k = 100
values = list(df_train_GD['价格'].values)
for i in range(90):
    nextvalue = autoRegression(values[-(k+i):],4,5)
    if(nextvalue == None): nextvalue = values[len(values)-1]
    values.append(nextvalue)
    

In [273]:
df_train_GD.index = df_train_GD['日期']

In [274]:
# 得到模型

# df_train_GD['2014-01-01':'2014-02-01']
# df_train_GD['2015-01-01':'2015-02-01']
# df_train_GD['2016-01-01':'2016-02-01']
year = 2016
x = []
y = []

for i in range(1,4):
    for d in range(1,32):
        if i == 2 and d > 28: continue
        tmp = []
        if d < 10: 
            s = str(year)+'-0'+str(i)+'-0'+str(d)
        else: 
            s = str(year)+'-0'+str(i)+'-'+str(d)
#         print(s)
        if len(df_train_GD[df_train_GD.index == s]['价格'].values) == 0:
            if len(y) == 0:
                y.append(df_train_GD[df_train_GD.index == '2015-01-01']['价格'].values[0])
            else:
                y.append(y[len(y)-1])
            
        else:
            y.append(df_train_GD[df_train_GD.index == s]['价格'].values[0])
        
        # 得到前一周的价格列表
        curtime = datetime.datetime.strptime(s,'%Y-%m-%d')
        lasttime = curtime - datetime.timedelta(days = 6)
        # 日期到字符串
        lasttime = datetime.datetime.strftime(lasttime,'%Y-%m-%d')
        curtime = datetime.datetime.strftime(curtime,'%Y-%m-%d')
        tmp.extend(list(df_train_GD[lasttime:curtime]['价格'].values))
        
        if len(tmp)<7:
            tmp.extend(tmp[len(tmp)-7:])
        
#         print(len(tmp))
        # 得到前两年对应时间的前5天，后五天
        if d < 10: 
            s = str(year-2)+'-0'+str(i)+'-0'+str(d)
        else: 
            s = str(year-2)+'-0'+str(i)+'-'+str(d)
        curtime = datetime.datetime.strptime(s,'%Y-%m-%d')
        lasttime = curtime - datetime.timedelta(days = 4)
        nexttime = curtime + datetime.timedelta(days = 4)
        
        curtime = datetime.datetime.strftime(curtime,'%Y-%m-%d')
        lasttime = datetime.datetime.strftime(lasttime,'%Y-%m-%d')
        nexttime = datetime.datetime.strftime(nexttime,'%Y-%m-%d')
        
        tmp.extend(list(df_train_GD[lasttime:curtime]['价格'].values))
        tmp.extend(list(df_train_GD[curtime:nexttime]['价格'].values))
                
        if len(tmp) < 17:
            tmp.extend(tmp[len(tmp)-17:])

        
            
        # 得到前一年对应时间的前5天，后五天
        if d < 10: 
            s = str(year-1)+'-0'+str(i)+'-0'+str(d)
        else: 
            s = str(year-1)+'-0'+str(i)+'-'+str(d)
        curtime = datetime.datetime.strptime(s,'%Y-%m-%d')
        lasttime = curtime - datetime.timedelta(days = 4)
        nexttime = curtime + datetime.timedelta(days = 4)
        
        curtime = datetime.datetime.strftime(curtime,'%Y-%m-%d')
        lasttime = datetime.datetime.strftime(lasttime,'%Y-%m-%d')
        nexttime = datetime.datetime.strftime(nexttime,'%Y-%m-%d')
        
        tmp.extend(list(df_train_GD[lasttime:curtime]['价格'].values))
        tmp.extend(list(df_train_GD[curtime:nexttime]['价格'].values))
        
        if len(tmp) < 27:
            tmp.extend(tmp[len(tmp)-27:])
            
#         print(len(tmp))
        x.append(tmp)
        
#         print(s)
        
# 得到参数
x = np.array(x)
y = np.array(y)

print(x.shape)

print(len(y))

from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(x, y)

regr.coef_


(90, 27)
90


array([ -6.43774508e-16,   1.49880108e-15,  -6.53521587e-16,
         6.32097263e-17,  -1.99143848e-16,  -6.69918386e-16,
         1.00000000e+00,  -2.47002296e-16,   5.21716133e-16,
        -1.18132539e-15,   1.13687307e-15,   1.24468947e-15,
        -1.25181297e-15,   6.06750501e-16,  -8.31042983e-16,
        -4.84352961e-16,   6.82290225e-16,   2.25514052e-17,
         7.19910243e-17,  -1.04950770e-16,   4.12864187e-16,
        -8.30065183e-16,  -7.22512328e-16,   4.65773253e-16,
         2.63677968e-16,  -1.49186219e-16,  -1.56125113e-16])

In [275]:
# 测试代码

print(df_train_GD[df_train_GD.index =='2016-01-01'])
print(df_train_GD['2016-01-01':'2016-01-20'])

# 得到前7天的时期
xtime = datetime.datetime.strptime('2016-01-10','%Y-%m-%d')
ytime = xtime - datetime.timedelta(days = 7)
# 日期到字符串
ytime = datetime.datetime.strftime(ytime,'%Y-%m-%d')


                    日期  地区   价格      数量    均重
日期                                           
2016-01-01  2016-01-01  广东  6.1  101.33  3.58
                    日期  地区    价格      数量    均重
日期                                            
2016-01-01  2016-01-01  广东  6.10  101.33  3.58
2016-01-02  2016-01-02  广东  6.12   87.39  3.51
2016-01-03  2016-01-03  广东  6.30   77.29  3.58
2016-01-04  2016-01-04  广东  6.27   74.43  3.60
2016-01-05  2016-01-05  广东  6.32   60.02  3.67
2016-01-06  2016-01-06  广东  6.21   47.99  3.63
2016-01-07  2016-01-07  广东  6.14   63.74  3.67
2016-01-08  2016-01-08  广东  5.92   76.50  3.56
2016-01-09  2016-01-09  广东  5.95   82.22  3.61
2016-01-10  2016-01-10  广东  5.82   82.78  3.63
2016-01-11  2016-01-11  广东  5.59   77.41  3.57
2016-01-12  2016-01-12  广东  5.46   63.88  3.59
2016-01-13  2016-01-13  广东  5.29   67.87  3.69
2016-01-14  2016-01-14  广东  5.16   59.52  3.68
2016-01-15  2016-01-15  广东  5.12   92.46  3.60
2016-01-16  2016-01-16  广东  4.97   76.66  3.58
2016-01-17  2016

In [279]:
# 得到模型

# df_train_GD['2014-01-01':'2014-02-01']
# df_train_GD['2015-01-01':'2015-02-01']
# df_train_GD['2016-01-01':'2016-02-01']
year = 2017
x = []
y = []

for i in range(1,4):
    for d in range(1,32):
        if i == 2 and d > 28: continue
        tmp = []
        if d < 10: 
            ts = str(year)+'-0'+str(i)+'-0'+str(d)
        else: 
            ts = str(year)+'-0'+str(i)+'-'+str(d)
        # 得到前一周的价格列表
        curtime = datetime.datetime.strptime(ts,'%Y-%m-%d')
        lasttime = curtime - datetime.timedelta(days = 6)
        # 日期到字符串
        lasttime = datetime.datetime.strftime(lasttime,'%Y-%m-%d')
        curtime = datetime.datetime.strftime(curtime,'%Y-%m-%d')
        tmp.extend(list(df_train_GD[lasttime:curtime]['价格'].values))
        
        if len(tmp)<7:
            tmp.extend(tmp[len(tmp)-7:])
        
#         print(len(tmp))
        # 得到前两年对应时间的前5天，后五天
        if d < 10: 
            s = str(year-2)+'-0'+str(i)+'-0'+str(d)
        else: 
            s = str(year-2)+'-0'+str(i)+'-'+str(d)
        curtime = datetime.datetime.strptime(s,'%Y-%m-%d')
        lasttime = curtime - datetime.timedelta(days = 4)
        nexttime = curtime + datetime.timedelta(days = 4)
        
        curtime = datetime.datetime.strftime(curtime,'%Y-%m-%d')
        lasttime = datetime.datetime.strftime(lasttime,'%Y-%m-%d')
        nexttime = datetime.datetime.strftime(nexttime,'%Y-%m-%d')
        
        tmp.extend(list(df_train_GD[lasttime:curtime]['价格'].values))
        tmp.extend(list(df_train_GD[curtime:nexttime]['价格'].values))
        
#         print(df_train_GD[curtime:nexttime])
        
        if len(tmp) < 17:
            tmp.extend(tmp[len(tmp)-17:])

        
            
        # 得到前一年对应时间的前5天，后五天
        if d < 10: 
            s = str(year-1)+'-0'+str(i)+'-0'+str(d)
        else: 
            s = str(year-1)+'-0'+str(i)+'-'+str(d)
        curtime = datetime.datetime.strptime(s,'%Y-%m-%d')
        lasttime = curtime - datetime.timedelta(days = 4)
        nexttime = curtime + datetime.timedelta(days = 4)
        
        curtime = datetime.datetime.strftime(curtime,'%Y-%m-%d')
        lasttime = datetime.datetime.strftime(lasttime,'%Y-%m-%d')
        nexttime = datetime.datetime.strftime(nexttime,'%Y-%m-%d')
        
        tmp.extend(list(df_train_GD[lasttime:curtime]['价格'].values))
        tmp.extend(list(df_train_GD[curtime:nexttime]['价格'].values))
        
#         print(df_train_GD[lasttime:curtime])
#         print(df_train_GD[curtime:nexttime])
        
        if len(tmp) < 27:
            tmp.extend(tmp[len(tmp)-27:])
        
#         print(tmp)
        tmp = np.array(tmp)
        
        
        y_ = regr.predict(tmp)
        
        t = pd.DataFrame({'日期':ts, '地区':None,'价格':y_,'数量':None,'均重':None})
        t.index = t['日期']
#         print(t)
        
        df_train_GD = df_train_GD.append(t)
        
        

KeyError: "Cannot get right slice bound for non-unique label: '2017-01-01'"

In [277]:
df_train_GD['2014-12-23':'2015-01-03']
df_train_GD.tail(5)

,价格,地区,均重,数量,日期
日期,,,,,
2017-03-27,4.95,None,NaN,NaN,2017-03-27
2017-03-28,4.95,None,NaN,NaN,2017-03-28
2017-03-29,4.95,None,NaN,NaN,2017-03-29
2017-03-30,4.95,None,NaN,NaN,2017-03-30
2017-03-31,4.95,None,NaN,NaN,2017-03-31


,日期,地区,价格,数量,均重
16300,2016-12-31,湖北,3.92,20.35,3.84
16301,2016-12-31,湖南,4.07,18.25,3.75
16302,2016-12-31,福建,4.59,10.37,3.63
16303,2016-12-31,贵州,4.95,3.92,4.74
16304,2016-12-31,重庆,4.55,4.47,4.94


In [284]:
for i in range(1,4):
    for d in range(1,32):
        if i == 2 and d > 28: 
            continue            
        print(d)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
